![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "Dados/UCI HAR Dataset/features.txt"
filename_labels = "Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze()
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze()
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze()
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [7]:
%%time
clf = DecisionTreeClassifier(ccp_alpha=0.001).fit(X_train, y_train)




CPU times: total: 4.94 s
Wall time: 4.95 s


In [9]:
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

print(f'Acurácia da melhor árvore na base de treino:    {train_score*100:.1f}')
print(f'Acurácia da melhor árvore na base de teste:     {test_score*100:.1f}')

Acurácia da melhor árvore na base de treino:    97.6
Acurácia da melhor árvore na base de teste:     88.3


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [12]:
%%time
prcomp = PCA(n_components=1).fit(X_train)
pc_train = prcomp.transform(X_train)
pc_test = prcomp.transform(X_test)


CPU times: total: 1.23 s
Wall time: 734 ms


In [13]:
clf = DecisionTreeClassifier(ccp_alpha=0.001).fit(pc_train, y_train)

train_score = clf.score(pc_train, y_train)
test_score = clf.score(pc_test, y_test)

print(f'Acurácia da melhor árvore na base de treino:    {train_score*100:.1f}')
print(f'Acurácia da melhor árvore na base de teste:     {test_score*100:.1f}')


Acurácia da melhor árvore na base de treino:    50.0
Acurácia da melhor árvore na base de teste:     45.7


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [33]:
prcomp_list = []
n_comp = [1, 2, 5, 10, 50]

for n in n_comp:
    start = time.time()
    prcomp = PCA(n_components=n).fit(X_train)
    pc_train = prcomp.transform(X_train)
    pc_test = prcomp.transform(X_test)
    
    clf = DecisionTreeClassifier(random_state=101, ccp_alpha=0.001).fit(pc_train, y_train)

    train_score = clf.score(pc_train, y_train)
    test_score = clf.score(pc_test, y_test)

    print(f'Acurácia da melhor árvore na base de treino para {n} componentes:    {train_score*100:.1f}')
    print(f'Acurácia da melhor árvore na base de teste {n} componentes:          {test_score*100:.1f}')
    print(f'Tempo de processamento: {time.time() - start}')
    print('-----')


Acurácia da melhor árvore na base de treino para 1 componentes:    50.0
Acurácia da melhor árvore na base de teste 1 componentes:          45.7
Tempo de processamento: 0.37638068199157715
-----
Acurácia da melhor árvore na base de treino para 2 componentes:    61.3
Acurácia da melhor árvore na base de teste 2 componentes:          58.5
Tempo de processamento: 0.2988700866699219
-----
Acurácia da melhor árvore na base de treino para 5 componentes:    84.6
Acurácia da melhor árvore na base de teste 5 componentes:          78.9
Tempo de processamento: 0.29853391647338867
-----
Acurácia da melhor árvore na base de treino para 10 componentes:    89.2
Acurácia da melhor árvore na base de teste 10 componentes:          82.4
Tempo de processamento: 0.4995238780975342
-----
Acurácia da melhor árvore na base de treino para 50 componentes:    91.6
Acurácia da melhor árvore na base de teste 50 componentes:          82.0
Tempo de processamento: 1.078035831451416
-----


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

**Resposta:** Na medida que o número de componentes foi aumentando, a acurácia melhorou e o tempo de processamento aumentou. Porém, 10 para 50 componentes houve uma leve queda na acurácia para a base teste.